In [113]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [52]:

csv_files = [
    '../scraping2/file1.csv',
    '../scraping2/file2.csv', 
    '../scraping2/file3.csv',
    '../scraping2/file4.csv'
]

dataframes = []
for file_path in csv_files:
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Loaded {file_path}: {df.shape[0]} rows")
        dataframes.append(df)
    else:
        print(f"File not found: {file_path}")


combined_df = pd.concat(dataframes, ignore_index=True)

original_rows = len(combined_df)
print(f"Combined DataFrame shape: {combined_df.shape}")
combined_df = combined_df.drop_duplicates()

combined_df.to_csv('masterdata.csv', index=False)

combined_df.head()


Loaded ../scraping2/file1.csv: 166850 rows
Loaded ../scraping2/file2.csv: 508100 rows


/var/folders/b1/kvks3q1d3118skc52j7g3pd40000gn/T/ipykernel_30157/1514448648.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded ../scraping2/file3.csv: 768400 rows
Loaded ../scraping2/file4.csv: 238850 rows
Combined DataFrame shape: (1682200, 8)


,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,Den procentuelle forskel mellem seneste udbudspris og salgsprisen %,Unnamed: 7
0,"Fritidshus FFritidshus Nordmandsvej 12, Lyngsb...",1.398.250 kr.,27-05-2025Fam. Salg,83 m² 16.846 kr/m²,4,1956,NaN,Aktuel værdi
1,Rækkehus RRækkehus Rosenlyparken 80 2670 Greve,2.620.000 kr.,27-05-2025Alm. Salg,120 m² 21.833 kr/m²,5,1972,-9%,Aktuel værdi
2,Landejendom LLandejendom Karlslunde Centervej ...,6.450.000 kr.,27-05-2025Alm. Salg,240 m² 26.875 kr/m²,6,1906,-2%,Aktuel værdi
3,Villa VVilla Hovedgaden 42 8763 Rask Mølle,460.000 kr.,27-05-2025Andet,176 m² 2.614 kr/m²,8,1908,NaN,Aktuel værdi
4,"Villa VVilla Kirkevej 5, Tjæreby 4230 Skælskør",2.200.000 kr.,27-05-2025Alm. Salg,211 m² 10.427 kr/m²,7,1780,-4%,Aktuel værdi


In [96]:

df=pd.read_csv("masterdata.csv")

/var/folders/b1/kvks3q1d3118skc52j7g3pd40000gn/T/ipykernel_30157/3202950873.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("masterdata.csv")


In [97]:
print(df.shape)
print(df.columns)

(1680333, 8)
Index(['Boligtype & Adresse', 'Købesum', 'Dato & Type', 'm² & Kr. / m²',
       'Vær.', 'Byggeår',
       'Den procentuelle forskel mellem seneste udbudspris og salgsprisen %',
       'Unnamed: 7'],
      dtype='object')


In [98]:
df = df.drop(columns=['Unnamed: 7', 'Den procentuelle forskel mellem seneste udbudspris og salgsprisen %'])


In [99]:
df["dato"] = df["Dato & Type"].str[0:10]
df["stype"] = df["Dato & Type"].str[10:].astype("category")

In [100]:
df["stype"].value_counts()

stype
Alm. Salg    1468715
Fam. Salg     132350
Andet          55154
Auktion        24090
-                 24
Name: count, dtype: int64

In [101]:
df.head()

,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,dato,stype
0,"Fritidshus FFritidshus Nordmandsvej 12, Lyngsb...",1.398.250 kr.,27-05-2025Fam. Salg,83 m² 16.846 kr/m²,4,1956,27-05-2025,Fam. Salg
1,Rækkehus RRækkehus Rosenlyparken 80 2670 Greve,2.620.000 kr.,27-05-2025Alm. Salg,120 m² 21.833 kr/m²,5,1972,27-05-2025,Alm. Salg
2,Landejendom LLandejendom Karlslunde Centervej ...,6.450.000 kr.,27-05-2025Alm. Salg,240 m² 26.875 kr/m²,6,1906,27-05-2025,Alm. Salg
3,Villa VVilla Hovedgaden 42 8763 Rask Mølle,460.000 kr.,27-05-2025Andet,176 m² 2.614 kr/m²,8,1908,27-05-2025,Andet
4,"Villa VVilla Kirkevej 5, Tjæreby 4230 Skælskør",2.200.000 kr.,27-05-2025Alm. Salg,211 m² 10.427 kr/m²,7,1780,27-05-2025,Alm. Salg


In [102]:
df = df.drop(columns=['Dato & Type'])

In [103]:
df["Købesum"] = df["Købesum"].str.replace("kr." , "")
df["Købesum"] = df["Købesum"].str.replace("." , "")
df["Købesum"] = df["Købesum"].str.strip() 
df["Købesum"] = pd.to_numeric(df["Købesum"].astype(np.int32))

print(df["Købesum"].head())


0    1398250
1    2620000
2    6450000
3     460000
4    2200000
Name: Købesum, dtype: int32


In [104]:
df["m2"]= df["m² & Kr. / m²"]
df["m2"] = df["m2"].str.split(' m²').str[0]
# Check what values can't be converted

df["m2"] = df["m2"].astype(np.float32)

print(df["m2"].head())

0     83.0
1    120.0
2    240.0
3    176.0
4    211.0
Name: m2, dtype: float32


In [105]:
parts = df["Boligtype & Adresse"].str.split("  ", expand=True)

df["btype"] = parts[0].str.split(' ').str[0].astype("category")

df["addresse"] = parts[0].str.split(' ').str[2:].str.join(" ")

zipcity = parts[1]
df["postnummer"] = zipcity.str[:4]  
df["by"] = zipcity.str[4:].str.strip()  

df.head(10)

,Boligtype & Adresse,Købesum,m² & Kr. / m²,Vær.,Byggeår,dato,stype,m2,btype,addresse,postnummer,by
0,"Fritidshus FFritidshus Nordmandsvej 12, Lyngsb...",1398250,83 m² 16.846 kr/m²,4,1956,27-05-2025,Fam. Salg,83.0,Fritidshus,"Nordmandsvej 12, Lyngsbæk",8400,Ebeltoft
1,Rækkehus RRækkehus Rosenlyparken 80 2670 Greve,2620000,120 m² 21.833 kr/m²,5,1972,27-05-2025,Alm. Salg,120.0,Rækkehus,Rosenlyparken 80,2670,Greve
2,Landejendom LLandejendom Karlslunde Centervej ...,6450000,240 m² 26.875 kr/m²,6,1906,27-05-2025,Alm. Salg,240.0,Landejendom,Karlslunde Centervej 76,4030,Tune
3,Villa VVilla Hovedgaden 42 8763 Rask Mølle,460000,176 m² 2.614 kr/m²,8,1908,27-05-2025,Andet,176.0,Villa,Hovedgaden 42,8763,Rask Mølle
4,"Villa VVilla Kirkevej 5, Tjæreby 4230 Skælskør",2200000,211 m² 10.427 kr/m²,7,1780,27-05-2025,Alm. Salg,211.0,Villa,"Kirkevej 5, Tjæreby",4230,Skælskør
5,Villa VVilla Kirkebakken 66 4621 Gadstrup,2951000,143 m² 20.636 kr/m²,5,1974,27-05-2025,Alm. Salg,143.0,Villa,Kirkebakken 66,4621,Gadstrup
6,"Landejendom LLandejendom Troldemosevej 3, Laug...",38000000,160 m² 237.500 kr/m²,6,1900,27-05-2025,Alm. Salg,160.0,Landejendom,"Troldemosevej 3, Laugø",3200,Helsinge
7,"Fritidshus FFritidshus Bøgetoft 16, Kelstrup S...",730000,40 m² 18.250 kr/m²,3,1966,27-05-2025,Alm. Salg,40.0,Fritidshus,"Bøgetoft 16, Kelstrup Strand",6100,Haderslev
8,Ejerlejlighed EEjerlejlighed Prins Valdemars A...,3125000,83 m² 37.651 kr/m²,2,2014,27-05-2025,Alm. Salg,83.0,Ejerlejlighed,Prins Valdemars Alle 9B,3450,Allerød
9,"Ejerlejlighed EEjerlejlighed Nordlandsgade 8, ...",4900000,82 m² 59.756 kr/m²,3,1911,27-05-2025,Alm. Salg,82.0,Ejerlejlighed,"Nordlandsgade 8, st. th",2300,København S


In [106]:
df = df.drop(columns=['Boligtype & Adresse', 'm² & Kr. / m²'])


In [107]:
df = df.drop_duplicates(subset=['addresse', 'postnummer', 'dato'], keep='first')

In [108]:
zd = pd.read_excel(r'DK_regions_zip_codes.xlsx')
zd["area"] = zd["area"].astype("category")
zd["region"] = zd["region"].astype("category")

In [109]:
def assign_region(row):
    postnr = int(row['postnummer'])
    
    if 0 <= postnr <= 999:
        return pd.Series(['Særlig for organisationer og store virksomheder', 'Ikke angivet'], index=['område', 'region'])
    elif 1000 <= postnr <= 2999:
        return pd.Series(['Hovedstaden, København', 'Sjælland'], index=['område', 'region'])
    elif 3000 <= postnr <= 3699:
        return pd.Series(['Nordsjælland', 'Sjælland'], index=['område', 'region'])
    elif 3700 <= postnr <= 3799:
        return pd.Series(['Bornholm', 'Bornholm'], index=['område', 'region'])
    elif 3800 <= postnr <= 3899:
        return pd.Series(['Færøerne', 'Færøerne'], index=['område', 'region'])
    elif 3900 <= postnr <= 3999:
        return pd.Series(['Grønland', 'Grønland'], index=['område', 'region'])
    elif 4000 <= postnr <= 4999:
        return pd.Series(['Andre øer', 'Sjælland'], index=['område', 'region'])
    elif 5000 <= postnr <= 5999:
        return pd.Series(['Fyn og øer', 'Fyn og øer'], index=['område', 'region'])
    elif 6000 <= postnr <= 6999:
        return pd.Series(['Sydjylland', 'Jylland'], index=['område', 'region'])
    elif 7000 <= postnr <= 7999:
        return pd.Series(['Sydjylland', 'Jylland'], index=['område', 'region'])
    elif 8000 <= postnr <= 8999:
        return pd.Series(['Øst- og Midtjylland', 'Jylland'], index=['område', 'region'])
    elif 9000 <= postnr <= 9999:
        return pd.Series(['Nordjylland', 'Jylland'], index=['område', 'region'])
    else:
        return pd.Series([None, None], index=['område', 'region'])

# Anvend funktionen på DataFrame
df[['område', 'region']] = df.apply(assign_region, axis=1)

In [131]:
print(df.shape)
df.head(10)

(1680312, 12)


,Købesum,Vær.,Byggeår,dato,stype,m2,btype,addresse,postnummer,by,område,region
0,1398250,4,1956,27-05-2025,Fam. Salg,83.0,Fritidshus,"Nordmandsvej 12, Lyngsbæk",8400,Ebeltoft,Øst- og Midtjylland,Jylland
1,2620000,5,1972,27-05-2025,Alm. Salg,120.0,Rækkehus,Rosenlyparken 80,2670,Greve,"Hovedstaden, København",Sjælland
2,6450000,6,1906,27-05-2025,Alm. Salg,240.0,Landejendom,Karlslunde Centervej 76,4030,Tune,Andre øer,Sjælland
3,460000,8,1908,27-05-2025,Andet,176.0,Villa,Hovedgaden 42,8763,Rask Mølle,Øst- og Midtjylland,Jylland
4,2200000,7,1780,27-05-2025,Alm. Salg,211.0,Villa,"Kirkevej 5, Tjæreby",4230,Skælskør,Andre øer,Sjælland
5,2951000,5,1974,27-05-2025,Alm. Salg,143.0,Villa,Kirkebakken 66,4621,Gadstrup,Andre øer,Sjælland
6,38000000,6,1900,27-05-2025,Alm. Salg,160.0,Landejendom,"Troldemosevej 3, Laugø",3200,Helsinge,Nordsjælland,Sjælland
7,730000,3,1966,27-05-2025,Alm. Salg,40.0,Fritidshus,"Bøgetoft 16, Kelstrup Strand",6100,Haderslev,Sydjylland,Jylland
8,3125000,2,2014,27-05-2025,Alm. Salg,83.0,Ejerlejlighed,Prins Valdemars Alle 9B,3450,Allerød,Nordsjælland,Sjælland
9,4900000,3,1911,27-05-2025,Alm. Salg,82.0,Ejerlejlighed,"Nordlandsgade 8, st. th",2300,København S,"Hovedstaden, København",Sjælland


In [124]:
def remove_outliers_iqr(df, column,multiplier=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
dfiqr = remove_outliers_iqr(df, 'Købesum')
print(dfiqr .shape)

(1528517, 12)


In [ ]:
def remove_outliers_isolation_forest(df, columns, contamination=0.1):

    data = df[columns].copy()
    data = data.dropna() 
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    
    
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    outlier_labels = iso_forest.fit_predict(data_scaled)
    
    mask = outlier_labels == 1
    return df[mask]

dff = remove_outliers_isolation_forest(df, ['Købesum', 'm2'])
print(dff.shape)

(1512290, 12)


In [134]:
def remove_outliers_manual(df):
    df = df[df['m2'] >= 10]  
    df = df[df['m2'] <= 1000]  
    
    df = df[df['Købesum'] >= 50000]  
    df = df[df['Købesum'] <= 30000000]  
    
    df = df[df['Byggeår'] >= 1800]  
    df = df[df['Byggeår'] <= 2025]  
    
    df = df[df['Vær.'] >= 1]  
    df = df[df['Vær.'] <= 15]  
    
    df['price_per_m2'] = df['Købesum'] / df['m2']
    df = df[df['price_per_m2'] >= 1000]   
    df = df[df['price_per_m2'] <= 250000]  
    
    df = df.drop('price_per_m2', axis=1)
    
    df= df[df['dato'] <= '30-12-1991']
    df= df[df['dato'] >= '01-01-2026']

    return df
dfm= remove_outliers_manual(df)
print(dfm.shape)

(1559197, 12)


In [125]:
dmfiqr=remove_outliers_iqr(dfm, 'Købesum',3)
print(dmfiqr.shape)

(1536961, 12)


In [128]:
dfmf=remove_outliers_isolation_forest(dfm, ['Købesum', 'm2'])
print(dfmf.shape)

(1434444, 12)


In [ ]:



#dfm['id'] = dfm.groupby(['addresse', 'postnummer']).ngroup()

#print(dfm[['addresse', 'postnummer', 'id']].head())
#print(f"Number of unique IDs: {dfm['id'].nunique()}")


                    addresse postnummer       id
0  Nordmandsvej 12, Lyngsbæk       8400   892869
1           Rosenlyparken 80       2670  1040999
2    Karlslunde Centervej 76       4030   624035
3              Hovedgaden 42       8763   528022
5             Kirkebakken 66       4621   644085
Number of unique IDs: 1559098


In [ ]:
#dfm.nunique()

Købesum        121300
Vær.               15
Byggeår           226
dato            10331
stype               5
m2                693
btype               5
addresse      1352278
postnummer        930
by                605
område              8
region              4
id            1559098
dtype: int64

In [ ]:
# Find rows where address is "bolbro sidevej 6"
#bolbro_rows = df[df['addresse'].str.lower() == 'bolbro sidevej 6']
#print(f"Found {len(bolbro_rows)} rows with address 'bolbro sidevej 6':")
#print(bolbro_rows[['addresse', 'postnummer', 'dato', 'Købesum','Vær.','m2','Byggeår']])


Found 1 rows with address 'bolbro sidevej 6':
                addresse postnummer        dato  Købesum  Vær.     m2  Byggeår
390993  Bolbro Sidevej 6       2960  11-05-2021  7495000     6  235.0     1982


In [ ]:
df = dfm.dropna()
print(df.shape)
df.to_csv('cleaned_masterdata.csv', index=False)

(1559197, 12)
